In [ ]:
"""
Authors      : Aditya Jain and Safwan Jamal
Date started : November 16, 2022
About        : Convex Optimization project; training script
"""

import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

from cifardataset import CIFARDataset

#### Loading Model

#### Loading Data

In [ ]:
train_set      = './image_list_cifar/train_full.csv'
num_epochs     = 1

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor()])
batch_size     = 32
class_list     = ['airplane', 'automobile', 'bird', 'cat', 'deer', 
                  'dog', 'frog', 'horse', 'ship', 'truck']
train_root_dir = './cifar-10-images/train/'
test_root_dir  = './cifar-10-images/test/'
test_set       = './image_list_cifar/test.csv'

train_data       = CIFARDataset(train_root_dir, train_set, class_list, transform)
train_dataloader = DataLoader(train_data,batch_size=batch_size, shuffle=True, num_workers=2)

test_data        = CIFARDataset(test_root_dir, test_set, class_list, transform)
test_dataloader  = DataLoader(test_data,batch_size=batch_size, shuffle=True, num_workers=2)

#### Training

In [ ]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

for epoch in range(num_epochs): 
    # Model Training
    model.train()
    for image_batch, label_batch in train_dataloader:    
        image_batch, label_batch = image_batch.to(device), label_batch.to(device)
        pass
    
    # Model Evaluation
    model.eval()
    for image_batch, label_batch in test_dataloader:    
        image_batch, label_batch = image_batch.to(device), label_batch.to(device)
        pass
